# Building with LangChain

## 0. Initial setup

In [1]:
# Install dependencies
%pip install --upgrade --quiet pip
%pip install --upgrade --quiet langchain langchain-openai langchainhub beautifulsoup4 faiss-cpu

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
# Set up Tavily and OpenAI API key environment variables
import getpass
import os

print("Enter Tavily API Key...")
os.environ["TAVILY_API_KEY"] = getpass.getpass()
print("Enter OpenAI API Key...")
os.environ["OPENAI_API_KEY"] = getpass.getpass()

Enter Tavily API Key...


 ········


Enter OpenAI API Key...


 ········


## 1. LLM Chain

In [3]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI()

In [4]:
llm.invoke("how can langsmith help with testing?")

AIMessage(content='Langsmith can help with testing in several ways:\n\n1. Language-specific test frameworks: Langsmith may provide specific frameworks or tools for testing code written in that language, making it easier to write and execute tests.\n\n2. Automated testing: Langsmith can help automate the testing process, allowing developers to quickly and easily run tests and identify issues in their code.\n\n3. Code analysis: Langsmith may offer tools for analyzing code and identifying potential bugs or vulnerabilities, helping developers improve the quality of their code.\n\n4. Integration with testing tools: Langsmith may integrate with popular testing tools and platforms, making it easier for developers to incorporate testing into their development workflow.\n\n5. Documentation: Langsmith may provide documentation and resources for writing effective tests and best practices for testing code in that language. \n\nOverall, Langsmith can support developers in creating robust and reliab

In [5]:
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a world class technical documentation writer."),
    ("user", "{input}")
])

In [6]:
# Now the fun part! We can now create our first "Chain"
chain = prompt | llm 

In [7]:
# And now we can invoke the new chain
chain.invoke({"input": "how can langsmith help with testing?"})

AIMessage(content='Langsmith can help with testing in several ways:\n\n1. Automated Testing: Langsmith can assist in writing automated test scripts in various programming languages to test software components and ensure they function correctly.\n\n2. Test Data Generation: Langsmith can generate test data sets to be used in testing scenarios, helping to cover a wide range of potential inputs and conditions.\n\n3. Test Case Management: Langsmith can help in creating and managing test cases, ensuring comprehensive test coverage and tracking the status of each test case.\n\n4. Performance Testing: Langsmith can assist in performance testing by running simulations and load tests to evaluate the performance of software under various conditions.\n\n5. Integration Testing: Langsmith can help in testing the integration of different software components to ensure they work seamlessly together.\n\nOverall, Langsmith can streamline the testing process, improve test coverage, and help identify and r

In [8]:
# Create a parser to help us clean up the output
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

In [9]:
# Now, extend the chain to include our parser (output will now be a string, instead of an AIMessage object)
chain = prompt | llm | output_parser

In [10]:
chain.invoke({"input": "how can langsmith help with testing?"})

'Langsmith can be a valuable tool for testing in several ways:\n\n1. Automated Testing: Langsmith can generate test cases automatically based on the specifications provided. This can help in creating comprehensive test suites that cover various scenarios and edge cases.\n\n2. Test Data Generation: Langsmith can assist in generating test data for different types of inputs, making it easier to test the functionality of a system under various conditions.\n\n3. Regression Testing: Langsmith can help in quickly generating new test cases or updating existing ones when changes are made to the system. This can streamline the process of regression testing and ensure that all critical functionalities are tested after each change.\n\n4. Test Coverage: Langsmith can analyze the coverage of test cases generated and identify any gaps in testing. This can help in ensuring thorough test coverage and reducing the risk of undetected issues.\n\n5. Integration Testing: Langsmith can aid in generating test

## 2. Retrieval chain

Next, we'll incorporate document retrieval, to make the model's responses more relevant

In [11]:
# Use WebBaseLoader to load relevant documents from the langchain docs!
from langchain_community.document_loaders import WebBaseLoader
loader = WebBaseLoader("https://docs.smith.langchain.com/user_guide")

docs = loader.load()

In [12]:
# We also need to generate embeddings (vector representations) of the user's input (and the LangChain docs) so we
# can determine which documents are relevant to the user's question. To do this, we need an embeddings model, so let's
# set one up.
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings()

In [13]:
# Now, we can split up the LangChain docs and represent vectors for each "chunk" of documentation
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents(docs)
vector = FAISS.from_documents(documents, embeddings)

In [14]:
# Now we create a prompt template. We will insert relevant documents into the <context></context>
# section of the prompt, which the LLM can use to answer the user's question.
from langchain.chains.combine_documents import create_stuff_documents_chain

prompt = ChatPromptTemplate.from_template("""Answer the following question based only on the provided context:

<context>
{context}
</context>

Question: {input}""")

document_chain = create_stuff_documents_chain(llm, prompt)

In [15]:
# Let's try it out! Here we provide a "dummy" document
from langchain_core.documents import Document

document_chain.invoke({
    "input": "how can langsmith help with testing?",
    "context": [Document(page_content="langsmith can let you visualize test results")]
})

'Langsmith can help by allowing you to visualize test results.'

In [16]:
# We will create a retrieval chain so we can fetch the *real* LangChain docs
from langchain.chains import create_retrieval_chain

retriever = vector.as_retriever()
retrieval_chain = create_retrieval_chain(retriever, document_chain)

In [17]:
# Now, we should get a *real*, relevant answer
response = retrieval_chain.invoke({"input": "how can langsmith help with testing?"})
print(response["answer"])

# LangSmith offers several features that can help with testing:...

LangSmith can help with testing by allowing developers to create datasets, run tests on LLM applications, upload test cases in bulk, create test cases on the fly, export test cases from application traces, and run custom evaluations to score test results. Additionally, LangSmith provides a comparison view for tracking and diagnosing regressions in test scores across multiple revisions of an application. The platform also offers a playground environment for rapid iteration and experimentation, logging every playground run for further analysis and comparison with other runs. LangSmith supports beta testing, feedback collection, annotation of traces, and adding runs to datasets to refine and improve application performance during testing phases.


## 3. Conversational retrieval chain

What if we want to have an ongoing conversation with the model, where it "remembers" our previous questions (and its answers)? Then we need to build a chatbot. Let's do that now.

In [18]:
# Now we set up a "history aware" retriever
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder

# First we need a prompt that we can pass into an LLM to generate this search query

prompt = ChatPromptTemplate.from_messages([
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
    ("user", "Given the above conversation, generate a search query to look up to get information relevant to the conversation")
])
retriever_chain = create_history_aware_retriever(llm, retriever, prompt)

In [19]:
# Let's set up a chat!
from langchain_core.messages import HumanMessage, AIMessage

chat_history = [HumanMessage(content="Can LangSmith help test my LLM applications?"), AIMessage(content="Yes!")]
retriever_chain.invoke({
    "chat_history": chat_history,
    "input": "Tell me how"
})

[Document(page_content='LangSmith User Guide | \uf8ffü¶úÔ∏è\uf8ffüõ†Ô∏è LangSmith', metadata={'source': 'https://docs.smith.langchain.com/user_guide', 'title': 'LangSmith User Guide | \uf8ffü¶úÔ∏è\uf8ffüõ†Ô∏è LangSmith', 'description': 'LangSmith is a platform for LLM application development, monitoring, and testing. In this guide, we‚Äôll highlight the breadth of workflows LangSmith supports and how they fit into each stage of the application development lifecycle. We hope this will inform users how to best utilize this powerful platform or give them something to consider if they‚Äôre just starting their journey.', 'language': 'en'}),
 Document(page_content='Skip to main contentLangSmith API DocsSearchGo to AppQuick StartUser GuideTracingEvaluationProduction Monitoring & AutomationsPrompt HubProxyPricingSelf-HostingCookbookUser GuideOn this pageLangSmith User GuideLangSmith is a platform for LLM application development, monitoring, and testing. In this guide, we‚Äôll highlight the bre

In [20]:
# Retrieval is working well, now let's set up a chain that will allow us to have an ongoing conversation
prompt = ChatPromptTemplate.from_messages([
    ("system", "Answer the user's questions based on the below context:\n\n{context}"),
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
])
document_chain = create_stuff_documents_chain(llm, prompt)
retrieval_chain = create_retrieval_chain(retriever_chain, document_chain)

In [21]:
# And test it...
chat_history = [HumanMessage(content="Can LangSmith help test my LLM applications?"), AIMessage(content="Yes!")]
retrieval_chain.invoke({
    "chat_history": chat_history,
    "input": "Tell me how"
})

{'chat_history': [HumanMessage(content='Can LangSmith help test my LLM applications?'),
  AIMessage(content='Yes!')],
 'input': 'Tell me how',
 'context': [Document(page_content='Skip to main contentLangSmith API DocsSearchGo to AppQuick StartUser GuideTracingEvaluationProduction Monitoring & AutomationsPrompt HubProxyPricingSelf-HostingCookbookUser GuideOn this pageLangSmith User GuideLangSmith is a platform for LLM application development, monitoring, and testing. In this guide, we‚Äôll highlight the breadth of workflows LangSmith supports and how they fit into each stage of the application development lifecycle. We hope this will inform users how to best utilize this powerful platform or give them something to consider if they‚Äôre just starting their journey.Prototyping‚ÄãPrototyping LLM applications often involves quick experimentation between prompts, model types, retrieval strategy and other parameters.\nThe ability to rapidly understand how the model is performing ‚Äî and debug

## 4. Agent

LangChain can also allow LLMs to invoke tools. Let's try that next.


In [22]:
# Set up a new retriever tool
from langchain.tools.retriever import create_retriever_tool

retriever_tool = create_retriever_tool(
    retriever,
    "langsmith_search",
    "Search for information about LangSmith. For any questions about LangSmith, you must use this tool!",
)

In [23]:
# We can now use Tavily (or another API service) to actively search for information
from langchain_community.tools.tavily_search import TavilySearchResults

search = TavilySearchResults()

In [24]:
# Generate a list of tools our agent can use (a single agent can call multiple tools, even other agents)
tools = [retriever_tool, search]

In [25]:
# Generate a new agent, using a prompt pulled from the LangChain Hub
from langchain_openai import ChatOpenAI
from langchain import hub
from langchain.agents import create_openai_functions_agent
from langchain.agents import AgentExecutor

# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/openai-functions-agent")

llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
agent = create_openai_functions_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [26]:
# Search Tavis for LangChain related docs on how it helps with testing
agent_executor.invoke({"input": "how can langsmith help with testing?"})



> Entering new AgentExecutor chain...

Invoking: `langsmith_search` with `{'query': 'how can LangSmith help with testing'}`


Skip to main contentLangSmith API DocsSearchGo to AppQuick StartUser GuideTracingEvaluationProduction Monitoring & AutomationsPrompt HubProxyPricingSelf-HostingCookbookUser GuideOn this pageLangSmith User GuideLangSmith is a platform for LLM application development, monitoring, and testing. In this guide, we‚Äôll highlight the breadth of workflows LangSmith supports and how they fit into each stage of the application development lifecycle. We hope this will inform users how to best utilize this powerful platform or give them something to consider if they‚Äôre just starting their journey.Prototyping‚ÄãPrototyping LLM applications often involves quick experimentation between prompts, model types, retrieval strategy and other parameters.
The ability to rapidly understand how the model is performing ‚Äî and debug where it is failing ‚Äî is incredibly important for

{'input': 'how can langsmith help with testing?',
 'output': 'LangSmith can help with testing in various ways throughout the application development lifecycle. Here are some key ways LangSmith can assist with testing:\n\n1. Prototyping: LangSmith supports quick experimentation between prompts, model types, retrieval strategies, and other parameters for LLM applications.\n2. Debugging: LangSmith provides clear visibility and debugging information at each step of an LLM sequence, making it easier to identify and root-cause issues when things go wrong.\n3. Initial Test Set: Developers can create datasets of inputs and reference outputs to run tests on LLM applications, allowing for test cases to be uploaded, created on the fly, or exported from application traces.\n4. Comparison View: LangSmith offers a user-friendly comparison view for test runs to track and diagnose regressions in test scores across multiple revisions of an application.\n5. Playground: LangSmith provides a playground en

In [27]:
# Note, our tool (Tavis) also has the ability to search for other types of information for us
agent_executor.invoke({"input": "what is the weather in SF?"})



> Entering new AgentExecutor chain...

Invoking: `tavily_search_results_json` with `{'query': 'weather in San Francisco'}`


[{'url': 'https://www.weatherapi.com/', 'content': "{'location': {'name': 'San Francisco', 'region': 'California', 'country': 'United States of America', 'lat': 37.78, 'lon': -122.42, 'tz_id': 'America/Los_Angeles', 'localtime_epoch': 1714999875, 'localtime': '2024-05-06 5:51'}, 'current': {'last_updated_epoch': 1714999500, 'last_updated': '2024-05-06 05:45', 'temp_c': 9.4, 'temp_f': 48.9, 'is_day': 0, 'condition': {'text': 'Partly cloudy', 'icon': '//cdn.weatherapi.com/weather/64x64/night/116.png', 'code': 1003}, 'wind_mph': 5.6, 'wind_kph': 9.0, 'wind_degree': 280, 'wind_dir': 'W', 'pressure_mb': 1021.0, 'pressure_in': 30.15, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 83, 'cloud': 25, 'feelslike_c': 7.7, 'feelslike_f': 45.8, 'vis_km': 16.0, 'vis_miles': 9.0, 'uv': 1.0, 'gust_mph': 11.3, 'gust_kph': 18.2}}"}, {'url': 'https://forecast.weather.gov/zipcity.

{'input': 'what is the weather in SF?',
 'output': 'The current weather in San Francisco is partly cloudy with a temperature of 48.9°F (9.4°C). The wind speed is 5.6 mph coming from the west, and the humidity is at 83%. The visibility is 9.0 miles, and the UV index is 1.0.'}

In [28]:
# We can also chat with the agent (agents can be chatbots)
chat_history = [HumanMessage(content="Can LangSmith help test my LLM applications?"), AIMessage(content="Yes!")]
agent_executor.invoke({
    "chat_history": chat_history,
    "input": "Tell me how"
})



> Entering new AgentExecutor chain...

Invoking: `langsmith_search` with `{'query': 'LangSmith LLM application testing services'}`


Skip to main contentLangSmith API DocsSearchGo to AppQuick StartUser GuideTracingEvaluationProduction Monitoring & AutomationsPrompt HubProxyPricingSelf-HostingCookbookUser GuideOn this pageLangSmith User GuideLangSmith is a platform for LLM application development, monitoring, and testing. In this guide, we‚Äôll highlight the breadth of workflows LangSmith supports and how they fit into each stage of the application development lifecycle. We hope this will inform users how to best utilize this powerful platform or give them something to consider if they‚Äôre just starting their journey.Prototyping‚ÄãPrototyping LLM applications often involves quick experimentation between prompts, model types, retrieval strategy and other parameters.
The ability to rapidly understand how the model is performing ‚Äî and debug where it is failing ‚Äî is incredibly import

{'chat_history': [HumanMessage(content='Can LangSmith help test my LLM applications?'),
  AIMessage(content='Yes!')],
 'input': 'Tell me how',
 'output': 'LangSmith provides a platform for LLM application development, monitoring, and testing. Here are some ways LangSmith can help test your LLM applications:\n\n1. Prototyping: LangSmith supports quick experimentation between prompts, model types, retrieval strategies, and other parameters for prototyping LLM applications.\n\n2. Debugging: LangSmith offers tracing capabilities to help developers debug issues in their LLM applications by providing clear visibility and debugging information at each step of an LLM sequence.\n\n3. Initial Test Set: Developers can create datasets of inputs and reference outputs to run tests on their LLM applications. LangSmith allows for custom evaluations to score test results.\n\n4. Comparison View: LangSmith provides a user-friendly comparison view for test runs to track and diagnose regressions in test sc

In [29]:
print("That's it! Pretty cool, right?")

That's it! Pretty cool, right?
